<a href="https://colab.research.google.com/github/jasonadu/gpt_in_practice/blob/main/3.%20%E5%88%9D%E8%AF%86LangChain%EF%BD%9C%E4%BD%A0%E7%9A%84%E7%91%9E%E5%A3%AB%E5%86%9B%E5%88%80%20-%20Qwen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 回顾

In [1]:
import os
from openai import OpenAI
from google.colab import userdata

client = OpenAI(
    api_key=userdata.get('DASHSCOPE_API_KEY'),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

def analyze_user_review(text):
    messages = []
    messages.append( {"role": "system",
              "content": """
              You are an assistant.
              Please, analyze the user reviews according to the following instruction:
              If the review is postive, you should output 'Y', otherwise output 'N'
              """})
    messages.append( {"role": "user", "content": text})

    response = client.chat.completions.create(
        model="qwen-plus",
        messages=messages
    )
    return response.choices[0].message.content


In [2]:
analyze_user_review("服务热情周到。")

'Y'

In [3]:
analyze_user_review("不理不睬。")

'N'

In [4]:
analyze_user_review("用餐环境差，等待时间长。")

'N'

In [5]:
!pip install langchain

In [6]:
!pip show langchain

Name: langchain
Version: 0.3.27
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: 


In [7]:
!pip install dashscope
!pip install -qU langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00


# 提示词模版

In [9]:
# https://python.langchain.com/docs/integrations/chat/tongyi/
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain_community.chat_models.tongyi import ChatTongyi

# prompt_template = "What is a good name for a company that makes {product}? And only return the best one."
prompt_template_str = "What is a good name for a company that makes {{product}}? And only return the best one."
prompt_template = PromptTemplate.from_template(prompt_template_str)

llm = ChatTongyi(model_name="qwen-plus", temperature=0, max_tokens=200,
                 dashscope_api_key=userdata.get('DASHSCOPE_API_KEY')) # dashscope_api_key is required

'''
LangChainDeprecationWarning:
The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
'''
# llm_chain = LLMChain(
#     llm=llm,
#     prompt=PromptTemplate.from_template(prompt_template)
# )
# llm_chain.run("colorful socks")

# 使用新的方式构建Runnable序列
llm_chain = prompt_template | llm

# 使用.invoke()方法运行链
response = llm_chain.invoke({"input": "colorful socks"})
print(response.content)
print(response)

Innovatech Solutions
content='Innovatech Solutions' additional_kwargs={} response_metadata={'model_name': 'qwen-plus', 'finish_reason': 'stop', 'request_id': '7d8ec099-ba40-97ea-9f57-38c0928ce594', 'token_usage': {'input_tokens': 32, 'output_tokens': 5, 'total_tokens': 37, 'prompt_tokens_details': {'cached_tokens': 0}}} id='run--1833f2db-47b7-415c-b2fa-9aa8a9706f44-0'


In [10]:
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt_template
)

products = [{"product":"'cloudnative devops platform'"},
            {"product":"'Noise cancellation headphone'"},
            {"product":"colorful socks"}]
llm_chain.apply(products)

/tmp/ipython-input-1953156135.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(


[{'text': 'InnovateCraft'}, {'text': 'Innovexa'}, {'text': 'InnovateCraft'}]

# API调用链
## HTTP request chain

In [ ]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chains import LLMRequestsChain
from langchain.chat_models import AzureChatOpenAI
# from langchain.chat_models import ChatOpenAI #直接访问OpenAI的GPT服务

#llm = ChatOpenAI(model_name="gpt-4", temperature=0) #直接访问OpenAI的GPT服务
llm = AzureChatOpenAI(deployment_name = deployment, model_name=model, temperature=0, max_tokens=200) #通过Azure的OpenAI服务

def query_baidu(question):
      template = """Between >>> and <<< are the raw search result text from web.
      Extract the answer to the question '{query}' or say "not found" if the information is not contained.
      Use the format
      Extracted:<answer or "not found">
      >>> {requests_result} <<<
      Extracted:"""

      PROMPT = PromptTemplate(
          input_variables=["query", "requests_result"],
          template=template,
      )

      inputs = {
          "query": question,
          "url": "http://www.baidu.com/s?wd=" + question.replace(" ", "+")
      }
      requests_chain = LLMRequestsChain(llm_chain = LLMChain(llm=llm, prompt=PROMPT), output_key="query_info", verbose=True)
      res = requests_chain.run(inputs)
      return res

In [ ]:
query_baidu("今天北京天气？")



> Entering new LLMRequestsChain chain...

> Finished chain.


'23°晴东南风1级20~26°C'

In [ ]:
# import os
# os.environ["SERPER_API_KEY"] = ""
# https://serper.dev
from langchain.utilities import GoogleSerperAPIWrapper
def query_web(question):
    search = GoogleSerperAPIWrapper()
    return search.run(question)


In [ ]:
query_web("今天北京天气？")

'72°F'

# 调用链

In [ ]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import SequentialChain
from langchain.chat_models import AzureChatOpenAI
# from langchain.chat_models import ChatOpenAI #直接访问OpenAI的GPT服务

#llm = ChatOpenAI(model_name="gpt-4", temperature=0) #直接访问OpenAI的GPT服务
llm = AzureChatOpenAI(deployment_name = deployment, model_name=model, temperature=0, max_tokens=200)

summarizing_prompt_template = """
Summarize the following content into a sentence less than 20 words:
---
{content}

"""
summarizing_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(summarizing_prompt_template),
    output_key = "summary"
)

translating_prompt_template = """
translate "{summary}" into Chinese:

"""

translating_chain = LLMChain(
    llm = llm,
    prompt=PromptTemplate.from_template(translating_prompt_template),
    output_key = "translated"
)

overall_chain = SequentialChain(
    chains=[summarizing_chain, translating_chain],
    input_variables=["content"],
    output_variables=[ "summary","translated"],
    verbose=True
)


In [ ]:
res = overall_chain("""
LangChain is a framework for developing applications powered by language models. It enables applications that are:

Data-aware: connect a language model to other sources of data
Agentic: allow a language model to interact with its environment
The main value props of LangChain are:

Components: abstractions for working with language models, along with a collection of implementations for each abstraction. Components are modular and easy-to-use, whether you are using the rest of the LangChain framework or not
Off-the-shelf chains: a structured assembly of components for accomplishing specific higher-level tasks
Off-the-shelf chains make it easy to get started. For more complex applications and nuanced use-cases, components make it easy to customize existing chains or build new ones.
""")

print("summary:"+res["summary"])

print("中文:"+res["translated"])



> Entering new SequentialChain chain...

> Finished chain.
summary:LangChain is a framework for creating applications powered by language models, offering data-aware and agentic features, modular components, and pre-assembled chains for specific tasks.
中文:"LangChain是一个用于创建由语言模型驱动的应用程序的框架，提供数据感知和代理特性，模块化组件，以及针对特定任务的预组装链条。"
